In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math
import copy

In [2]:
"""
By Lauren Schmutz (contact: lschmutz@andrew.cmu.edu). Pushed to github July 14, 2023.
Notes:
1.  Model does not experience significant efficiency drawbacks. Should run well with
    100+ periods; but arbitrage concerns exist. Note the arbitrage detector builtin.
2.  Class structure in a barebones state. No inheritance is used.
    Instead a "wrapper" model class stores the stock and interest models as
    a base, and a bond class is enabled to calculated two-coin price via method.
    This way, multiple bonds can be calculated and compared using the same stock
    and rate model, and new models may be initialized as well.
3.  Check input variables and sample numbers for how to use the model. For "call list",
    an input of None means no call option at that period. Otherwise, use a numerical
    input to represent the strike price at that time. For convertible and distress lists,
    input 0 means the stock is not convertible/never in distress at that period. Note
    further that these inputs expect the same number of elements in the list as there are
    periods in the model (including 0, i.e. t+1), even though having such optionality at time 0
    may not make sense. There are no failsafe mechanisms that prevent users
    from these inputs, though, and a user will encounter errors if they use them
    improperly. In the distress list, the program expects tuples of
    (distress price, coupon multiplier) at all periods except the final one, which expects
    (distress price, coupon multiplier, face multiplier).
"""

'\nBy Lauren Schmutz (contact: lschmutz@andrew.cmu.edu). Pushed to github July 14, 2023.\nNotes:\n1.  Model does not experience significant efficiency drawbacks. Should run well with\n    100+ periods; but arbitrage concerns exist. Note the arbitrage detector builtin.\n2.  Class structure in a barebones state. No inheritance is used.\n    Instead a "wrapper" model class stores the stock and interest models as\n    a base, and a bond class is enabled to calculated two-coin price via method.\n    This way, multiple bonds can be calculated and compared using the same stock\n    and rate model, and new models may be initialized as well.\n3.  Check input variables and sample numbers for how to use the model. For "call list",\n    an input of None means no call option at that period. Otherwise, use a numerical\n    input to represent the strike price at that time. For convertible and distress lists,\n    input 0 means the stock is not convertible/never in distress at that period. Note\n    f

In [3]:
def displayChart(tab):
    Chart=[]
    columns = []
    N = len(tab)-1
    for i in range(N, -1, -1):
        chart = []
        columns.append(N-i)
        for j in range(0,i):
            chart.append("")
        for j in range(i, N+1):
            entry = tab[j][i]
            chart.append(entry)
        Chart.append(chart)
    df = pd.DataFrame(Chart, columns = columns)
    return df.style.hide_index()

In [4]:
def list_to_dict(L):
    res = {}
    for i in range(len(L)):
        res[i] = L[i]
    return res

In [5]:
def tree(t, init, up, down, negative_allowed):
    """t=num of time periods. init=initial value. up, down factors : num, negative values allowed : bool."""
    res = {}
    gap = up+down
    if negative_allowed:
        for i in range(t+1):
            res[i] = np.arange(init-i*down, init+(i+1)*up, gap)
    else:
        for i in range(t+1):
            res[i] = np.maximum(np.zeros(i+1), np.arange(init-i*down, init+(i+1)*up, gap))
    return res

In [6]:
# def bond_tree(bond):
#     print("unimplemented")
#     L = []
#     for i in range(bond.periods):
#         pass

In [7]:
class HoLee:
    def __init__(self, periods, R0, lam, sig):
        self.periods = periods
        self.R0 = R0
        self.lam = lam
        self.sig = sig
        self.tree_exists = False

    def create_tree(self):
        if self.lam+self.sig > 0:
            self.up = self.lam+self.sig
            self.down = self.sig-self.lam
        else:
            self.up = self.sig-self.lam
            self.down = self.lam+self.sig
        
        self.tree = tree(self.periods, self.R0, self.up, self.down, True)
        self.tree_exists = True
        return self.tree

In [8]:
class Stocks:
    def __init__(self, periods, S0, alpha, beta):
        self.periods = periods
        self.S0 = S0
        self.alpha = alpha
        self.up = self.alpha
        self.beta = beta
        self.down = self.beta
        self.tree_exists = False

    def create_tree(self):
        self.tree = tree(self.periods, self.S0, self.alpha, self.beta, False)
        self.tree_exists = True
        return self.tree

In [9]:
class Model:
    """Model contains objects from HoLee, Stocks. Creation of trees are delayed via methods to address potential efficiency concerns."""
    def __init__(self, periods):
        self.periods = periods
        self.stocks_exist = False
        self.rates_exist = False
        self.arbitrage = False
    
    def init_stocks(self, S0, alpha, beta):
        self.stocks_obj = Stocks(self.periods, S0, alpha, beta)
        self.stock_up = alpha
        self.stock_down = beta
        self.stocks_tree = self.stocks_obj.create_tree()
        self.stocks_exist = True

    def init_HoLee(self, R0, lam, sig):
        self.rates_obj = HoLee(self.periods, R0, lam, sig)
        if lam+sig > 0:
            self.rate_up = lam+sig
            self.rate_down = sig-lam
        else:
            self.rate_up = sig-lam
            self.rate_down = lam+sig
        self.rates_tree = self.rates_obj.create_tree()
        self.rates_exist = True

In [10]:
def rate_p_values(model):
    t = model.periods
    res = {}
    for i in range(t):
        res[i] = np.full(i+1, 0.5)
    res[t] = np.ones(t+1)
    # model.rate_p_vals = res
    return res

In [11]:
def p_values(model):
    t = model.periods
    if not (model.stocks_exist and model.rates_exist):
            print("need to initialize stocks and/or rates")
            return None
    stock_price_chart = model.stocks_tree
    rates = model.rates_tree
    P = {i: np.ones(i+1) for i in range (t+1)}
    # P = {i: [1]*(i+1) for i in range(t+1)}
    for i in range(0, t):
        for j in range(i+1):
            if stock_price_chart[i][j] == 0:
                P[i][j] = -1
            else:
                d = stock_price_chart[i+1][j] / stock_price_chart[i][j]
                u = stock_price_chart[i+1][j+1] / stock_price_chart[i][j]
                p_soon = (1 + rates[i][j] - d) / (u-d)
                if (p_soon<0) or (p_soon>1):
                    model.arbitrage = True
                P[i][j] = p_soon
    return P

In [12]:
class Bond:
    def __init__(self, model, face, q, distr_list, conv_list, call_list):
        self.model = model
        self.periods = self.model.periods
        self.face = face
        self.q = q
        self.coup = self.face * self.q
        self.distr_dict = list_to_dict(distr_list)
        self.conv_dict = list_to_dict(conv_list)
        self.call_dict = list_to_dict(call_list)
        self.p_vals_exist = False

    def init_p_vals(self):
        if not (self.model.stocks_exist or self.model.rates_exist):
            print("need to initialize stocks and rates in outer model. do so and try again")
            return False
        elif not self.model.stocks_exist:
            print("need to initialize stocks in outer model. do so and try again")
            return False
        elif not self.model.rates_exist:
            print("need to initialize rates in outer model. do so and try again")
            return False
        else:
            self.p_vals = p_values(self.model)
            self.rate_p_vals = rate_p_values(self.model)
            if self.model.arbitrage: print("WARNING: Arbitrage exists in model.")
            return True
    
    def one_coin(self):
        print("unimplemented")
        if not self.init_p_vals():
            return False
        face = self.face
        t = self.model.periods
        S = self.model.stocks_tree
        R = self.model.rates_tree
        S_p = self.p_vals
        R_p = self.rate_p_vals
        coup = self.coup
        res = {}
        return None


# Idea: one dictionary. Key: integer tuple (time period, # stock heads, # tails), Value: bond price before coupon.
#                                                                             or: value: [bond price b4 coup, coup at time]
# WARNING: call formula not currently implemented

    def two_coin(self):
        if not self.init_p_vals:
            return False
        # self.p_vals = p_values(self.model)
        # self.rate_p_vals = rate_p_values(self.model)
        face = self.face
        t = self.model.periods
        S = self.model.stocks_tree
        R = self.model.rates_tree
        S_p = self.p_vals
        R_p = self.rate_p_vals
        coup = self.coup
        res = {}
        #fill in the last period 
        cur_call = self.call_dict[t]
        for i in range(t+1):            #i: number of stock heads
            for j in range(t+1):        #j: number of rate heads
                cur_stock = S[t][i]
                res[(t, i, j)] = [0,0]
                if cur_stock > 0:
                    cur_exp = face * (self.distr_dict[t][2] if cur_stock <= self.distr_dict[t][0] else 1)
                    if cur_call != None:
                        cur_exp = min(cur_exp, cur_call)
                    res[(t, i, j)][0] = max(self.conv_dict[t]*cur_stock, cur_exp)
                res[(t, i, j)][1] = coup if cur_stock > self.distr_dict[t][0] else (self.distr_dict[t][1] * coup if cur_stock > 0 else 0)
                # res[(t, i, j)] = max(conv_list[t]*cur_stock, face * (bond.distr_list[t+1][2] if cur_stock <= bond.distr_list[t+1][0] else 1))
        #now all the final period prices have been filled. what remains is to perform backwards induction
        #Backwards Induction:
        for n in range(t-1, -1, -1):
            cur_call = self.call_dict[n]
            for i in range(n+1):        #i: number of stock heads
                for j in range(n+1):    #j: number of rate heads
                    cur_stock = S[n][i]
                    cur_rate = R[n][j]
                    cur_disc = 1/(1+cur_rate)
                    #p values
                    s_h_p = S_p[n][i]
                    s_t_p = 1 - s_h_p
                    r_h_p = R_p[n][j]
                    r_t_p = 1 - r_h_p
                    #there are four cases. two adjacent stock price moves and two adjacent rates moves. make sure to get the correct pvals
                    #stock head rate head
                    hh = res[(n+1, i+1, j+1)][0]
                    hh_c = res[(n+1, i+1, j+1)][1]
                    #stock head rate tail
                    ht = res[(n+1, i+1, j)][0]
                    ht_c = res[(n+1, i+1, j)][1]
                    #stock tail rate head
                    th = res[(n+1, i, j+1)][0]
                    th_c = res[(n+1, i, j+1)][1]
                    #stock tail rate tail
                    tt = res[(n+1, i, j)][0]
                    tt_c = res[(n+1, i, j)][1]
                    #fill formula
                    cur_exp = 0 if cur_stock == 0 else cur_disc * (s_h_p*r_h_p*(hh+hh_c) + s_h_p*r_t_p*(ht+ht_c) + s_t_p*r_h_p*(th+th_c) + s_t_p*r_t_p*(tt+tt_c))
                    #call
                    if cur_call != None:
                        cur_exp = min(cur_exp, cur_call)
                    res[(n, i, j)] = [0,0]
                    res[(n, i, j)][0] = cur_exp
                    res[(n, i, j)][1] = coup if cur_stock > self.distr_dict[n][0] else (self.distr_dict[n][1] * coup if cur_stock > 0 else 0)
                    if n == 0: res[(n, i, j)][1] = 0
        if self.model.arbitrage: print("WARNING: Arbitrage exists in model.")
        return res


In [13]:
t_global = 10
model = Model(t_global)
model.init_stocks(30,5,5)
print("stocks")
displayChart(model.stocks_tree)

stocks


0,1,2,3,4,5,6,7,8,9,10
,,,,,,,,,,80.000000
,,,,,,,,,75.000000,70.000000
,,,,,,,,70.000000,65.000000,60.000000
,,,,,,,65.000000,60.000000,55.000000,50.000000
,,,,,,60.000000,55.000000,50.000000,45.000000,40.000000
,,,,,55.000000,50.000000,45.000000,40.000000,35.000000,30.000000
,,,,50.000000,45.000000,40.000000,35.000000,30.000000,25.000000,20.000000
,,,45.000000,40.000000,35.000000,30.000000,25.000000,20.000000,15.000000,10.000000
,,40.000000,35.000000,30.000000,25.000000,20.000000,15.000000,10.000000,5.000000,0.000000
,35.000000,30.000000,25.000000,20.000000,15.000000,10.000000,5.000000,0.000000,0.000000,0.000000


In [14]:
model.init_HoLee(0.04, 0.001, 0.001)
print("interest rates")
displayChart(model.rates_tree)

interest rates


0,1,2,3,4,5,6,7,8,9,10
,,,,,,,,,,0.060000
,,,,,,,,,0.058000,0.058000
,,,,,,,,0.056000,0.056000,0.056000
,,,,,,,0.054000,0.054000,0.054000,0.054000
,,,,,,0.052000,0.052000,0.052000,0.052000,0.052000
,,,,,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
,,,,0.048000,0.048000,0.048000,0.048000,0.048000,0.048000,0.048000
,,,0.046000,0.046000,0.046000,0.046000,0.046000,0.046000,0.046000,0.046000
,,0.044000,0.044000,0.044000,0.044000,0.044000,0.044000,0.044000,0.044000,0.044000
,0.042000,0.042000,0.042000,0.042000,0.042000,0.042000,0.042000,0.042000,0.042000,0.042000


In [15]:
distr_list = [(10, 0) for i in range(t_global + 1)]
distr_list[t_global] = (10, 0, 0.5)
conv_list = np.ones(t_global+1)
call_list = np.full(t_global+1, None)
cbond1 = Bond(model, 50, 0.04, distr_list, conv_list, call_list)
cbond1.init_p_vals()
print("p tilde")
displayChart(cbond1.p_vals)

p tilde


0,1,2,3,4,5,6,7,8,9,10
,,,,,,,,,,1.000000
,,,,,,,,,0.935000,1.000000
,,,,,,,,0.892000,0.864000,1.000000
,,,,,,,0.851000,0.824000,0.797000,1.000000
,,,,,,0.812000,0.786000,0.760000,0.734000,1.000000
,,,,,0.775000,0.750000,0.725000,0.700000,0.675000,1.000000
,,,,0.740000,0.716000,0.692000,0.668000,0.644000,0.620000,1.000000
,,,0.707000,0.684000,0.661000,0.638000,0.615000,0.592000,0.569000,1.000000
,,0.676000,0.654000,0.632000,0.610000,0.588000,0.566000,0.544000,0.522000,1.000000
,0.647000,0.626000,0.605000,0.584000,0.563000,0.542000,0.521000,-1.000000,-1.000000,1.000000


In [16]:
print("rate p tilde")
displayChart(cbond1.rate_p_vals)

rate p tilde


0,1,2,3,4,5,6,7,8,9,10
,,,,,,,,,,1.000000
,,,,,,,,,0.500000,1.000000
,,,,,,,,0.500000,0.500000,1.000000
,,,,,,,0.500000,0.500000,0.500000,1.000000
,,,,,,0.500000,0.500000,0.500000,0.500000,1.000000
,,,,,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000
,,,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000
,,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000
,,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000
,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,1.000000


In [17]:
D = cbond1.two_coin()
print(D[(0,0,0)])

[50.787239788287145, 0]


In [18]:
def sim_two_coin(t, vol, rate, rate_vol):
    t_global = t
    model = Model(t_global)
    model.init_stocks(30,vol,vol)

    model.init_HoLee(rate, rate_vol, rate_vol)

    distr_list = [(t_global, 0) for i in range(t_global + 1)]
    distr_list[t_global] = (t_global, 0, 0.5)
    conv_list = np.ones(t_global+1)
    call_list = np.full(t_global+1, None)
    cbond1 = Bond(model, 50, rate, distr_list, conv_list, call_list)
    cbond1.init_p_vals()
    D = cbond1.two_coin()
    return D[(0,0,0)][0]

sim_two_coin(10, 5, 0.04, 0.001)

50.787239788287145